# 🏦 贷款审批数据集 - 完整探索性数据分析 (EDA)

## 📋 目标
1. 理解数据集结构和质量
2. 分析各个特征的分布
3. 探索特征与贷款批准的关系
4. 发现业务洞察
5. 为机器学习建模做准备

## 📊 数据来源
- **Dataset**: Loan Approval Prediction Dataset
- **Source**: Kaggle (architsharma01)
- **Size**: ~4,000-45,000 records
- **Task**: Binary Classification (Approved/Rejected)

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 设置可视化风格
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# 中文字体支持 (如果需要)
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # Mac
# plt.rcParams['axes.unicode_minus'] = False

print("✅ 库导入成功！")

## 📂 1. 数据加载与基本信息

In [ ]:
# 数据路径
data_path = Path("../data/raw")

# 查找CSV文件
csv_files = list(data_path.glob("*.csv"))
print(f"找到的CSV文件: {csv_files}")

if not csv_files:
    print("⚠️ 未找到数据文件！")
    print("请按照 DATA_SETUP.md 的指引下载数据集")
    print("\n快速下载命令:")
    print("  python scripts/download_data.py")
else:
    # 加载第一个CSV文件
    df = pd.read_csv(csv_files[0])
    print(f"\n✅ 成功加载数据: {csv_files[0].name}")
    print(f"📊 数据集大小: {df.shape[0]} 行, {df.shape[1]} 列")

In [ ]:
# 查看前几行数据
print("📋 前5行数据预览:")
df.head()

In [ ]:
# 数据集基本信息
print("ℹ️ 数据集详细信息:")
df.info()

In [ ]:
# 数值型特征的统计描述
print("📈 数值型特征统计描述:")
df.describe()

In [ ]:
# 分类型特征的统计描述
print("📊 分类型特征统计描述:")
df.describe(include='object')

## 🔍 2. 数据质量检查

In [ ]:
# 缺失值分析
print("🕵️ 缺失值检查:")
missing_data = pd.DataFrame({
    '列名': df.columns,
    '缺失值数量': df.isnull().sum().values,
    '缺失值百分比': (df.isnull().sum().values / len(df) * 100).round(2)
})
missing_data = missing_data[missing_data['缺失值数量'] > 0].sort_values('缺失值数量', ascending=False)

if len(missing_data) > 0:
    print(missing_data.to_string(index=False))
else:
    print("✅ 没有缺失值！")

In [ ]:
# 重复值检查
duplicates = df.duplicated().sum()
print(f"\n📝 重复行数: {duplicates}")
if duplicates > 0:
    print(f"⚠️ 重复行占比: {(duplicates/len(df)*100):.2f}%")
else:
    print("✅ 没有重复行！")

In [ ]:
# 数据类型检查
print("\n🔤 各列数据类型:")
data_types = pd.DataFrame({
    '列名': df.columns,
    '数据类型': df.dtypes.values,
    '唯一值数量': [df[col].nunique() for col in df.columns],
    '示例值': [str(df[col].iloc[0]) for col in df.columns]
})
print(data_types.to_string(index=False))

## 🎯 3. 目标变量分析 - Loan Status

In [ ]:
# 假设目标变量是 'loan_status' 或类似的列名
# 根据实际数据调整列名
target_col = None
for possible_name in ['loan_status', 'Loan_Status', 'status', 'approval_status', 'approved']:
    if possible_name in df.columns:
        target_col = possible_name
        break

if target_col:
    print(f"🎯 目标变量: {target_col}")
    print(f"\n目标变量分布:")
    target_counts = df[target_col].value_counts()
    print(target_counts)
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 计数图
    target_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
    axes[0].set_title(f'{target_col} Distribution (Count)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Status', fontsize=12)
    axes[0].set_ylabel('Count', fontsize=12)
    axes[0].tick_params(axis='x', rotation=45)
    
    # 添加数值标签
    for i, v in enumerate(target_counts):
        axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')
    
    # 饼图
    target_pct = df[target_col].value_counts(normalize=True) * 100
    axes[1].pie(target_pct, labels=target_pct.index, autopct='%1.1f%%',
                colors=['#2ecc71', '#e74c3c'], startangle=90)
    axes[1].set_title(f'{target_col} Distribution (Percentage)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ 贷款批准率: {target_pct.iloc[0]:.2f}%")
    print(f"❌ 贷款拒绝率: {target_pct.iloc[1]:.2f}%")
    
    # 检查类别不平衡
    imbalance_ratio = target_counts.max() / target_counts.min()
    print(f"\n⚖️ 类别不平衡比例: {imbalance_ratio:.2f}:1")
    if imbalance_ratio > 2:
        print("⚠️ 数据集存在类别不平衡，建议使用SMOTE或调整类别权重")
else:
    print("⚠️ 未找到目标变量列，请检查数据集")
    print(f"可用列: {df.columns.tolist()}")

## 📊 4. 数值型特征分布分析

In [ ]:
# 识别数值型列
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"📊 数值型特征 ({len(numeric_cols)}个): {numeric_cols}")

# 为每个数值型特征创建分布图
if len(numeric_cols) > 0:
    n_cols = 3
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for idx, col in enumerate(numeric_cols):
        if idx < len(axes):
            axes[idx].hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'{col} Distribution', fontweight='bold')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frequency')
            
            # 添加统计信息
            mean_val = df[col].mean()
            median_val = df[col].median()
            axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
            axes[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
            axes[idx].legend()
    
    # 隐藏多余的子图
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 箱线图检测异常值
if len(numeric_cols) > 0:
    n_cols = 3
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for idx, col in enumerate(numeric_cols):
        if idx < len(axes):
            sns.boxplot(data=df, y=col, ax=axes[idx], color='skyblue')
            axes[idx].set_title(f'{col} - Outlier Detection', fontweight='bold')
            axes[idx].set_ylabel(col)
            
            # 计算异常值数量
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)][col]
            axes[idx].text(0, df[col].max(), f'Outliers: {len(outliers)}', 
                          ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat'))
    
    # 隐藏多余的子图
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 🏷️ 5. 分类型特征分析

In [ ]:
# 识别分类型列
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
# 排除目标变量
if target_col and target_col in categorical_cols:
    categorical_cols.remove(target_col)

print(f"🏷️ 分类型特征 ({len(categorical_cols)}个): {categorical_cols}")

if len(categorical_cols) > 0:
    n_cols = 2
    n_rows = (len(categorical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for idx, col in enumerate(categorical_cols):
        if idx < len(axes):
            value_counts = df[col].value_counts()
            value_counts.plot(kind='barh', ax=axes[idx], color=sns.color_palette('Set2'))
            axes[idx].set_title(f'{col} Distribution', fontweight='bold', fontsize=12)
            axes[idx].set_xlabel('Count', fontsize=10)
            axes[idx].set_ylabel(col, fontsize=10)
            
            # 添加数值标签
            for i, v in enumerate(value_counts):
                axes[idx].text(v + 10, i, str(v), va='center', fontweight='bold')
    
    # 隐藏多余的子图
    for idx in range(len(categorical_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 🔗 6. 特征相关性分析

In [ ]:
# 数值型特征相关性热图
if len(numeric_cols) > 1:
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[numeric_cols].corr()
    
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # 找出高度相关的特征对
    print("\n🔍 高度相关的特征对 (|r| > 0.7):")
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                high_corr_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    correlation_matrix.iloc[i, j]
                ))
    
    if high_corr_pairs:
        for feat1, feat2, corr in high_corr_pairs:
            print(f"  {feat1} <-> {feat2}: {corr:.3f}")
    else:
        print("  没有发现高度相关的特征对")

## 🎯 7. 特征与目标变量的关系分析

In [ ]:
# 数值型特征与目标变量的关系
if target_col and len(numeric_cols) > 0:
    n_cols = 3
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for idx, col in enumerate(numeric_cols):
        if idx < len(axes):
            sns.boxplot(data=df, x=target_col, y=col, ax=axes[idx], palette='Set2')
            axes[idx].set_title(f'{col} by {target_col}', fontweight='bold')
            axes[idx].set_xlabel(target_col)
            axes[idx].set_ylabel(col)
            axes[idx].tick_params(axis='x', rotation=45)
    
    # 隐藏多余的子图
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 分类型特征与目标变量的关系
if target_col and len(categorical_cols) > 0:
    n_cols = 2
    n_rows = (len(categorical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and n_cols == 1 else axes
    
    for idx, col in enumerate(categorical_cols):
        if idx < len(axes):
            # 创建交叉表
            ct = pd.crosstab(df[col], df[target_col], normalize='index') * 100
            ct.plot(kind='bar', ax=axes[idx], stacked=False, 
                   color=['#2ecc71', '#e74c3c'], alpha=0.8)
            axes[idx].set_title(f'{target_col} Rate by {col}', fontweight='bold')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Percentage (%)')
            axes[idx].legend(title=target_col)
            axes[idx].tick_params(axis='x', rotation=45)
    
    # 隐藏多余的子图
    for idx in range(len(categorical_cols), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

## 💡 8. 业务洞察分析

In [ ]:
# 根据实际列名调整以下分析
# 这里提供通用的业务分析框架

print("💡 业务洞察分析")
print("=" * 50)

# 1. 贷款金额分析
loan_amount_cols = [col for col in df.columns if 'loan' in col.lower() and 'amount' in col.lower()]
if loan_amount_cols:
    loan_col = loan_amount_cols[0]
    print(f"\n1️⃣ 贷款金额分析 ({loan_col}):")
    print(f"   平均贷款额: ${df[loan_col].mean():,.2f}")
    print(f"   中位数贷款额: ${df[loan_col].median():,.2f}")
    print(f"   最大贷款额: ${df[loan_col].max():,.2f}")
    print(f"   最小贷款额: ${df[loan_col].min():,.2f}")

# 2. 收入分析
income_cols = [col for col in df.columns if 'income' in col.lower()]
if income_cols:
    income_col = income_cols[0]
    print(f"\n2️⃣ 收入分析 ({income_col}):")
    print(f"   平均收入: ${df[income_col].mean():,.2f}")
    print(f"   中位数收入: ${df[income_col].median():,.2f}")
    
    # 如果同时有贷款金额，计算贷款收入比
    if loan_amount_cols:
        loan_to_income = df[loan_col] / df[income_col]
        print(f"   平均贷款/收入比: {loan_to_income.mean():.2f}")

# 3. 信用分数分析
credit_cols = [col for col in df.columns if 'credit' in col.lower() or 'score' in col.lower()]
if credit_cols:
    credit_col = credit_cols[0]
    print(f"\n3️⃣ 信用分数分析 ({credit_col}):")
    print(f"   平均信用分: {df[credit_col].mean():.0f}")
    print(f"   中位数信用分: {df[credit_col].median():.0f}")
    
    # 信用分等级分布
    def credit_category(score):
        if pd.isna(score):
            return 'Unknown'
        elif score >= 750:
            return 'Excellent (750+)'
        elif score >= 700:
            return 'Good (700-749)'
        elif score >= 650:
            return 'Fair (650-699)'
        elif score >= 600:
            return 'Poor (600-649)'
        else:
            return 'Very Poor (<600)'
    
    credit_categories = df[credit_col].apply(credit_category)
    print(f"\n   信用等级分布:")
    print(credit_categories.value_counts().to_string())

print("\n" + "=" * 50)

In [ ]:
# 批准率按不同维度分析
if target_col:
    print("\n📊 不同维度的贷款批准率分析")
    print("=" * 50)
    
    # 按信用分等级
    if credit_cols:
        credit_col = credit_cols[0]
        credit_categories = df[credit_col].apply(credit_category)
        approval_by_credit = pd.crosstab(credit_categories, df[target_col], normalize='index') * 100
        print("\n按信用等级的批准率:")
        print(approval_by_credit.to_string())
    
    # 按就业状态
    employment_cols = [col for col in df.columns if 'employ' in col.lower()]
    if employment_cols:
        emp_col = employment_cols[0]
        approval_by_emp = pd.crosstab(df[emp_col], df[target_col], normalize='index') * 100
        print("\n按就业状态的批准率:")
        print(approval_by_emp.to_string())
    
    print("\n" + "=" * 50)

## 📝 9. 关键发现总结

In [ ]:
print("\n" + "="*60)
print("📝 EDA 关键发现总结")
print("="*60)

print("\n✅ 数据质量:")
print(f"   - 数据集大小: {df.shape[0]} 行, {df.shape[1]} 列")
print(f"   - 缺失值: {df.isnull().sum().sum()} 个")
print(f"   - 重复行: {df.duplicated().sum()} 行")

if target_col:
    print(f"\n🎯 目标变量 ({target_col}):")
    print(f"   - 类别分布: {df[target_col].value_counts().to_dict()}")
    approval_rate = (df[target_col].value_counts(normalize=True).iloc[0] * 100)
    print(f"   - 批准率: {approval_rate:.2f}%")

print(f"\n📊 特征概况:")
print(f"   - 数值型特征: {len(numeric_cols)} 个")
print(f"   - 分类型特征: {len(categorical_cols)} 个")

print("\n💡 建议:")
print("   1. 特征工程:")
print("      - 创建贷款/收入比特征")
print("      - 对分类变量进行编码 (One-Hot 或 Label Encoding)")
print("      - 考虑特征标准化/归一化")
print("   2. 模型选择:")
print("      - XGBoost (推荐): 适合表格数据，可处理非线性关系")
print("      - Random Forest: 稳健且可解释")
print("      - Logistic Regression: 作为基线模型")
print("   3. 评估指标:")
print("      - Accuracy, Precision, Recall, F1-Score")
print("      - ROC-AUC: 评估分类器性能")
print("      - 混淆矩阵: 理解错误类型")

print("\n" + "="*60)
print("✅ EDA 完成！可以开始模型训练了。")
print("="*60)

## 💾 10. 保存处理后的数据

In [ ]:
# 保存清洗后的数据到 processed 目录
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

output_path = processed_dir / "loan_data_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"✅ 清洗后的数据已保存到: {output_path}")
print(f"📊 保存的数据大小: {df.shape[0]} 行, {df.shape[1]} 列")